# Configuração e Carregamento de Dados

In [14]:
!pip install pyspark # Instala o PySpark no ambiente Colab

In [15]:
import pyspark # Importa a biblioteca PySpark
from pyspark.sql import SparkSession # Importa a classe SparkSession, usada para iniciar a sessão Spark
from pyspark.sql.functions import col, concat, lit # Importa funções SQL essenciais: col (referência à coluna), concat (para concatenar strings) e lit (para adicionar um valor literal/constante)

In [16]:
spark = SparkSession.builder.getOrCreate() # Cria ou obtém uma instância do SparkSession

In [17]:
from google.colab import drive # Importa o módulo drive do Google Colab
drive.mount('/content/drive') # Monta o Google Drive para permitir a leitura dos arquivos

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
produtos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/produtos.csv", header=True, inferSchema=True) # Lê o DataFrame produtos do caminho especificado no Drive
clientes = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/clientes.csv", header=True, inferSchema=True) # Lê o DataFrame clientes
itens_pedido = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/itens_pedido.csv", header=True, inferSchema=True) # Lê o DataFrame itens_pedido
pedidos = spark.read.csv("/content/drive/MyDrive/Material de apoio - M27/pedidos.csv", header=True, inferSchema=True) # Lê o DataFrame pedidos

# Operações de Junção (Join)

In [19]:
join_pedidos_clientes_df = pedidos.join(clientes, pedidos['id_cliente'] == clientes['id_cliente'], 'inner') # Combina o DataFrame pedidos com clientes,e o tipo de junção é inner (retorna apenas linhas que têm correspondência em ambos os DataFrames)
join_pedidos_clientes_df = pedidos.join(clientes, 'id_cliente') # Junção Simplificada (Padrão): Refaz a junção. Quando apenas o nome da coluna é fornecido ('id_cliente'), o PySpark assume que a coluna existe em ambos os DataFrames e realiza um inner join por padrão

# Lista os tipos de junção mais comuns no Spark
"""
inner # Retorna apenas as linhas que têm correspondência em ambos os DataFrames
outer # Retorna todas as linhas de ambos os DataFrames, incluindo as que não têm correspondência
left_outer # Retorna todas as linhas do DataFrame da esquerda (pedidos) e as correspondentes do DataFrame da direita (clientes)
right_outter # Retorna todas as linhas do DataFrame da direita (clientes) e as correspondentes do DataFrame da esquerda (pedidos)
"""

pedidos.show(5) # Exibe as 5 primeiras linhas de pedidos
clientes.show(5) # Exibe as 5 primeiras linhas de clientes
join_pedidos_clientes_df.show(5) # Exibe as 5 primeiras linhas do DataFrame resultante da junção

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|           id_pedido|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|    delivered|2017-10-02 10:56:33|  2017-10-02 11:07:15|      2017-10-04 19:55:00| 2017-10-10 21:25:13|  2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|    delivered|2018-07-24 20:41:37|  2018-07-26 03:24:27|      2018-07-26 14:31:00| 2018-08-07 15:27:45|  2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|    delivered|2018-08-08 08:38:49|  2018-08-08 08:55:23|      2018-08-08 13:50:00| 2018-08-17 18:06:29|  2018-09

In [20]:
join_itens_pedido_df = itens_pedido.join(pedidos, 'id_pedido').join(produtos, 'id_produto') # Executa duas junções consecutivas (em cadeia). Combina itens_pedido com pedidos pela chave id_pedido, e o resultado é imediatamente combinado com produtos pela chave id_produto
join_itens_pedido_df.show() # Exibe o resultado da junção de três DataFrames.

+--------------------+--------------------+--------------+--------------------+-------------------+------+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio| preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+------+-----------+----------------

In [21]:
join_pedidos_entregues_df = pedidos.filter(col('status_pedido') == 'canceled').join(clientes, 'id_cliente') # Primeiro, filtra o DataFrame pedidos para incluir apenas aqueles com status_pedido igual a 'canceled' e, em seguida, combina o resultado filtrado com o DataFrame clientes
join_pedidos_entregues_df.show() # xibe o resultado da junção, que contém apenas pedidos cancelados e seus dados de cliente

+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|           id_pedido|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+-----------+--------------------+--------------+
|6d6b50b66d79f8082...|1b9ecfe83cdc25925...|     canceled|2018-08-04 14:29:27|  2018-08-07 04:10:26|                     NULL|                NULL|  2018-08-14 00:00:00|8ea097b1824dbd4d1...|       5514|           sao paulo|            SP|
|e3fe72696c4713d64...|714fb133a6730ab81...|     

# Manipulação e Reorganização de Colunas

In [22]:
pedidos_concat_df = pedidos.withColumn('id_pedido_cliente', concat(col('id_pedido'), lit('-'), col('id_cliente'))) # Cria uma nova coluna chamada id_pedido_cliente. O valor dessa coluna é a concatenação do id_pedido, o literal (string constante) '-', e o id_cliente

colunas = pedidos_concat_df.columns # Obtém uma lista dos nomes de todas as colunas do DataFrame.
print(colunas) # Imprime a lista de colunas (a nova coluna id_pedido_cliente estará no final)

colunas.remove('id_pedido_cliente') # Remove o nome da nova coluna da sua posição atual (final da lista)
colunas.insert(0, 'id_pedido_cliente') # Insere o nome da coluna id_pedido_cliente na posição 0 (início da lista)

pedidos_reorganizar_df = pedidos_concat_df.select(colunas) # Cria um novo DataFrame (pedidos_reorganizar_df) selecionando as colunas na ordem definida pela lista Python colunas, reorganizando o DataFrame
pedidos_reorganizar_df.show(n=5, truncate=False) # Exibe o resultado da reorganização, mostrando a nova coluna como a primeira

['id_pedido', 'id_cliente', 'status_pedido', 'data_compra_pedido', 'data_aprovacao_pedido', 'data_envio_transportadora', 'data_entrega_cliente', 'data_estimada_entrega', 'id_pedido_cliente']
+-----------------------------------------------------------------+--------------------------------+--------------------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|id_pedido_cliente                                                |id_pedido                       |id_cliente                      |status_pedido|data_compra_pedido |data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|
+-----------------------------------------------------------------+--------------------------------+--------------------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+
|e481f51cbdc54678b7cc49136f2d6af

# Escrita em Parquet e Encerramento

In [23]:
join_itens_pedido_df.write.mode('overwrite').option('header', 'true').parquet('/content/drive/MyDrive/Material de apoio - M27/join_itens_pedido.parquet') # Salva o resultado da junção de três tabelas no formato Parquet. Usa o modo overwrite para sobrescrever o diretório, se necessário

In [24]:
spark.read.option('header', 'true').parquet('/content/drive/MyDrive/Material de apoio - M27/join_itens_pedido.parquet').show(5) # Lê o arquivo Parquet recém-salvo de volta no Spark e exibe as 5 primeiras linhas para verificar a escrita

+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+--------------------+-------------+-------------------+---------------------+-------------------------+--------------------+---------------------+--------------------+--------------------+-------------------------+------------------------+--------------+----------------------+-----------------+------------------+
|          id_produto|           id_pedido|item_id_pedido|         id_vendedor|  data_limite_envio|preco|valor_frete|          id_cliente|status_pedido| data_compra_pedido|data_aprovacao_pedido|data_envio_transportadora|data_entrega_cliente|data_estimada_entrega|   categoria_produto|tamanho_nome_produto|tamanho_descricao_produto|quantidade_fotos_produto|peso_produto_g|comprimento_produto_cm|altura_produto_cm|largura_produto_cm|
+--------------------+--------------------+--------------+--------------------+-------------------+-----+-----------+-------------------

In [25]:
spark.stop() # Encerra o SparkSession e libera os recursos